In [2]:
import cPickle as pickle
data=pickle.load( open( "data/train10.pkl", "rb" ))

In [ ]:
class HMM:
        def __init__(self, state_list, observation_list,
                 transition_proba = None,
                 observation_proba = None,
                 initial_state_proba = None, smoothing_obs = 0.01):
          
            print "HMM creating with: "
            self.N = len(state_list)       # number of states
            self.M = len(observation_list) # number of possible emissions
            print str(self.N)+" states"
            print str(self.M)+" observations"
            
            self.omega_Y = state_list
            self.omega_Y.append("*")
            self.omega_X = observation_list
            
            if transition_proba is None:
                self.transition_proba = zeros( (self.N+1, self.N+1, self.N), float) 
            else:
                self.transition_proba=transition_proba
            if observation_proba is None:
                self.observation_proba = zeros( (self.M, self.N), float) 
            else:
                self.observation_proba=observation_proba
            """if initial_state_proba is None:
                self.initial_state_proba = zeros( (self.N,), float ) 
            else:
                self.initial_state_proba=initial_state_proba
            """
            self.make_indexes() # build indexes, i.e the mapping between token and int
            self.smoothing_obs = smoothing_obs 
            
        def make_indexes(self):
            """Creates the reverse table that maps states/observations names
            to their index in the probabilities array"""
            self.Y_index = {}
            for i in range(self.N+1):
                self.Y_index[self.omega_Y[i]] = i
                
            self.X_index = {}
            for i in range(self.M):
                self.X_index[self.omega_X[i]] = i
      
        def get_observationIndices( self, observations ):
            """return observation indices, i.e 
            return [self.O_index[o] for o in observations]
            and deals with OOVs
            """
            indices = zeros( len(observations), int )
            k = 0
            for o in observations:
                if o in self.X_index:
                    indices[k] = self.X_index[o]
                k += 1
            return indices

    
        def data2indices(self, sent): 
            """From a word of the corpus: 
            - extract the letter and coorection 
            - returns two list of indices, one for each
            -> (letterid, correctionid)
            """
            letterids = list()
            correctionids  = list()
            for couple in sent:
                ltr = couple[0]
                crt = couple[1]
                letterids.append(self.X_index[ltr])
                correctionids.append(self.Y_index[crt])
            return letterids,correctionids
            
        def observation_estimation(self, pair_counts):
            """ Build the observation distribution: 
                observation_proba is the observation probablility matrix
                    [b_ki],  b_ki = Pr(X_t=v_k|Y_t=q_i)"""
            
            
            
            
            
        def transition_estimation(self, trans_counts):
            """ Build the transition distribution: 
                transition_proba is the transition matrix with : 
                [a_ij] a[i,j] = Pr(Y_(t+1)=q_i|Y_t=q_j)
            """
            # fill with counts
            for pair in trans_counts:
                i=self.Y_index[pair[1]]
                j=self.Y_index[pair[0]]
                self.transition_proba[i,j]=trans_counts[pair]
            # normalize
            self.transition_proba=self.transition_proba/self.transition_proba.sum(axis=0).reshape(1,self.N)
        
        def init_estimation(self, init_counts):
            """Build the init. distribution"""
            # fill with counts
            for tag in init_counts:
                i=self.Y_index[tag]
                self.initial_state_proba[i]=init_counts[tag]
            # normalize
            self.initial_state_proba=self.initial_state_proba/sum(self.initial_state_proba)
             
        
        def supervised_training(self, pair_counts, trans_counts,init_counts):
            """ Train the HMM's parameters. This function wraps everything"""
            self.observation_estimation(pair_counts)
            self.transition_estimation(trans_counts)
            self.init_estimation(init_counts)

In [8]:
def make_counts(corpus):
    """ 
    Build different count tables to train a HMM. Each count table is a dictionnary. 
    Returns: 
    * c_letter: letter counts
    * c_correction: correction counts
    * c_pairs: count of pairs (letter,correction)
    
    * c_bitag: count of tag bigram 
    * c_tritag: count of tag trigram 
    * c_inits: count of tag found in the first position
    
    """
    c_letter = dict()
    c_correction = dict()
    c_pairs= dict()
    c_bitag = dict()
    c_tritag = dict()
    c_inits = dict()
    
    
    for word in corpus:
        for i in range(len(word)):
            couple= word[i]
            letter = couple[0]
            correction = couple[1]
            #Counting letter 
            if letter in c_letter:
                c_letter[letter] +=1
            else:
                c_letter[letter] =1  
            #Counting correction
            if correction in c_correction:
                c_correction[correction] +=1
            else:
                c_correction[correction] =1
            #Counting par(letter, correction)
            if couple in c_pairs:
                c_pairs[couple] +=1
            else :
                c_pairs[couple] =1
            #Counting bitag(corr_i, corr_(i+1))
            if i > 0:
                bitag = (word[i-1][1], correction)
                if bitag in c_bitag:
                    c_bitag[bitag] += 1
                else:
                    c_bitag[bitag] =1
                    
            #Counting tritag
            if i > 1:
                tritag = (word[i-2][1],word[i-1][1], correction)
                if tritag in c_tritag :
                    c_tritag[tritag] +=1
                else :
                    c_tritag[tritag] =1
                    
            if i == 0:
                if correction in c_inits:
                    c_inits[correction] +=1
                else :
                    c_inits[correction] =1
    return c_letter, c_correction, c_pairs, c_bitag, c_tritag, c_inits

26